# 1.
Напоследок посмотрите, как продолжительность заездов распределяется по девяти сетям («Большая восьмёрка» и «Другие»). Загвоздка может быть вот в чём: если продолжительность сильно различается, то сравнивать показатели разных сетей будет неправильно. Например, если в сети больше заправок продолжительностью по 60–70 секунд, чем в других, это может понижать медианное значение. Чтобы проверить, не происходит ли такое, сгруппируйте данные из good_data по group_name и постройте гистограммы. Первым делом создайте столбец для группировки.
Выполните следующие шаги:
Создайте столбец group_name в таблице good_data так же, как делали раньше в station_stat_full.
Выведите на экран первые 5 строк good_data, чтобы проверить работу нового столбца.

Подсказка
Скопируйте метод where(), который вы применяли к station_stat_full, и просто замените station_stat_full на good_data.

In [3]:
import pandas as pd

data = pd.read_csv('../datasets/visits.csv', sep='\t')

# фильтруем слишком быстрые и медленные заезды и АЗС
data['too_fast'] = data['time_spent'] < 60
data['too_slow'] = data['time_spent'] > 1000
too_fast_stat = data.pivot_table(index='id', values='too_fast')
good_ids = too_fast_stat.query('too_fast < 0.5')
good_data = data.query('id in @good_ids.index')
good_data = good_data.query('60 <= time_spent <= 1000')

# считаем данные по отдельным АЗС и по сетям
station_stat = data.pivot_table(index='id', values='time_spent', aggfunc='median')
good_stations_stat = good_data.pivot_table(index='id', values='time_spent', aggfunc='median')
stat = data.pivot_table(index='name', values='time_spent')
good_stat = good_data.pivot_table(index='name', values='time_spent', aggfunc='median')
stat['good_time_spent'] = good_stat['time_spent']

id_name = good_data.pivot_table(index='id', values='name', aggfunc=['first', 'count'])
id_name.columns = ['name', 'count']
station_stat_full = id_name.join(good_stations_stat)

# считаем показатели сетей из показателей АЗС,
# а не усреднённые заезды на все АЗС сети
good_stat2 = (
    station_stat_full
    .query('count > 30')
    .pivot_table(index='name', values='time_spent', aggfunc=['median', 'count'])
)
good_stat2.columns = ['median_time', 'stations']
final_stat = stat.join(good_stat2)

big_nets_stat = final_stat.query('stations > 10')
station_stat_full['group_name'] = (
    station_stat_full['name']
    .where(station_stat_full['name'].isin(big_nets_stat.index), 'Другие')
)

stat_grouped = (
    station_stat_full
    .query('count > 30')
    .pivot_table(index='group_name', values='time_spent', aggfunc=['median', 'count'])
)
stat_grouped.columns = ['time_spent', 'count']

good_data['group_name'] = good_data['name'].where(good_data['name'].isin(big_nets_stat.index), 'Другие')
print(good_data.head())

         date_time        id  time_spent     name  too_fast  too_slow  \
0  20180406T165358  76144fb2        98.0  Василёк     False     False   
2  20180403T172824  76144fb2       220.0  Василёк     False     False   
6  20180402T082321  76144fb2       555.0  Василёк     False     False   
7  20180407T003408  76144fb2       286.0  Василёк     False     False   
9  20180405T131939  76144fb2       248.0  Василёк     False     False   

  group_name  
0    Василёк  
2    Василёк  
6    Василёк  
7    Василёк  
9    Василёк  


# 2.
Теперь, когда есть столбец group_name, сгруппируйте good_data и постройте гистограмму, чтобы увидеть, как распределяется продолжительность заездов в каждой сети.
Выполните следующие шаги:
Сгруппируйте good_data по group_name, используя цикл for. Используйте в цикле переменные name и group_data.
В каждой итерации group_data вызывайте метод hist(), чтобы построить гистограмму по значениям time_spent на 50 корзин.

Подсказка
Начните цикл с for name, group_data. Методом groupby() сделайте срез из good_data по group_name. Примените hist() к group_data. Первым укажите столбец time_spent вторым — параметр bins=.

In [ ]:
for name, group_data in good_data.groupby('group_name'):
    group_data.plot(kind='hist',
                    y='time_spent',
                    title=name,
                    bins=50)